In [1]:
from random import seed
import requests

url = "https://freewebnovel.com/lord-of-the-mysteries.html"
session_obj = requests.Session()
response = session_obj.get(url, headers={"User-Agent": "Mozilla/5.0"})

print(response.content)

b'<!DOCTYPE html>\n<html lang="en-US">\n<head>\n    <meta charset="UTF-8">\n    <title>Lord of the Mysteries Novel | Free Web Novel</title>\n    <meta name="description" content="Read Lord of the Mysteries novel online free...Lord of the Mysteries is genre Xuanhuan novel by author Cuttlefish That Loves Diving. Alternative names: Lord of Mysteries, Lord Of Enigma, \xe8\xaf\xa1\xe7\xa7\x98\xe4\xb9\x8b\xe4\xb8\xbb.">\n    <meta name="image" content="https://freewebnovel.com/files/article/image/0/494/494s.jpg">\n    <meta name="keywords" content="Lord of the Mysteries, Xuanhuan novel, latest chapter, novel updates, author Cuttlefish That Loves Diving,\xe7\x88\xb1\xe6\xbd\x9c\xe6\xb0\xb4\xe7\x9a\x84\xe4\xb9\x8c\xe8\xb4\xbc">\n    <meta property="og:site_name" content="freewebnovel.com"/>\n    <meta property="og:type" content="website" />\n    <meta property="og:title" content="Lord of the Mysteries">\n    <meta property="og:image" content="https://freewebnovel.com/files/article/image/0/494/

In [10]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response.content,'html.parser')
print(soup)

<!DOCTYPE html>

<html lang="en-US">
<head>
<meta charset="utf-8"/>
<title>Lord of the Mysteries Novel | Free Web Novel</title>
<meta content="Read Lord of the Mysteries novel online free...Lord of the Mysteries is genre Xuanhuan novel by author Cuttlefish That Loves Diving. Alternative names: Lord of Mysteries, Lord Of Enigma, 诡秘之主." name="description"/>
<meta content="https://freewebnovel.com/files/article/image/0/494/494s.jpg" name="image"/>
<meta content="Lord of the Mysteries, Xuanhuan novel, latest chapter, novel updates, author Cuttlefish That Loves Diving,爱潜水的乌贼" name="keywords"/>
<meta content="freewebnovel.com" property="og:site_name">
<meta content="website" property="og:type">
<meta content="Lord of the Mysteries" property="og:title"/>
<meta content="https://freewebnovel.com/files/article/image/0/494/494s.jpg" property="og:image"/>
<meta content="https://freewebnovel.com/lord-of-the-mysteries.html" property="og:url"/>
<meta content="Lord of the Mysteries -  In the waves of 

In [23]:

# Вырезаем часть, в которой находятся ссылки на главы 
# при этом указаны ссылки не полностью, а только хвост, например "...href="/invincible-novel/chapter-1.html..."
a = soup.find_all("a", class_="con")
#print(a[3].get('href'))
#print(a[3])
print(a[3].get('title'))

soup2 = BeautifulSoup(str(a),'html.parser')

# Получение хвостов ссылок на главы книги
soup_links = soup2.find_all("a", class_="con")

# Проверка:
#soup_links[0].get('href')

Chapter 1429 - In Modern Day 27


In [12]:
#novel_link = 'https://freewebnovel.com/invincible-novel.html'
novel_link = 'https://freewebnovel.com/lord-of-the-mysteries.html'
#Переменная либо для всех, либо для N последних, либо K первых глав



In [13]:
import requests 
import numpy as np
import io
import os
from bs4 import BeautifulSoup
from FB2 import FictionBook2, Author
from urllib import request
import time, sys
from IPython.display import clear_output

In [9]:
def update_progress(progress):
    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

In [14]:
# Получаем html-текст страницы
r = requests.get(novel_link)
html_text = r.text
soup = BeautifulSoup(html_text,'html.parser')
print(html_text)

<!DOCTYPE html><html lang="en-US"><head><title>Just a moment...</title><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta http-equiv="X-UA-Compatible" content="IE=Edge"><meta name="robots" content="noindex,nofollow"><meta name="viewport" content="width=device-width,initial-scale=1"><style>*{box-sizing:border-box;margin:0;padding:0}html{line-height:1.15;-webkit-text-size-adjust:100%;color:#313131;font-family:system-ui,-apple-system,BlinkMacSystemFont,Segoe UI,Roboto,Helvetica Neue,Arial,Noto Sans,sans-serif,Apple Color Emoji,Segoe UI Emoji,Segoe UI Symbol,Noto Color Emoji}body{display:flex;flex-direction:column;height:100vh;min-height:100vh}.main-content{margin:8rem auto;max-width:60rem;padding-left:1.5rem}@media (width <= 720px){.main-content{margin-top:4rem}}.h2{font-size:1.5rem;font-weight:500;line-height:2.25rem}@media (width <= 720px){.h2{font-size:1.25rem;line-height:1.5rem}}#challenge-error-text{background-image:url();background-repeat:no-repeat;background-s

In [8]:

# Вырезаем часть, в которой находятся ссылки на главы 
# при этом указаны ссылки не полностью, а только хвост, например "...href="/invincible-novel/chapter-1.html..."
a = soup.find_all("ul", class_="ul-list5")
soup2 = BeautifulSoup(str(a),'html.parser')

# Получение хвостов ссылок на главы книги
soup_links = soup2.find_all("a", class_="con")

# Проверка:
#soup_links[0].get('href')

IndexError: list index out of range

In [6]:

# Собираем список с реальными ссылками на главы книги
i = 1
chapter_link = []
garbage = []
for chapter in soup_links:
    href_link_string = chapter.get('href')
    chapter_last_link = href_link_string[18:] # вырезаем из текста "/invincible-novel/chapter-3753.html" крайнее значение "hapter-3753.html"
    chapter_link.append(novel_link[:-5]+"/"+chapter_last_link)

    garbage.append(href_link_string)

# В общем случе раскоментить
# Вырзем первые 6 ссылок из-за указания последних загруженных глав
#chapter_link = chapter_link[6:]

# Выгрузить последние главы
chapter_link = chapter_link[-200:]

In [7]:
chapter_soup_text4

NameError: name 'chapter_soup_text4' is not defined

In [411]:
# Проходимся по главам и достаем оттуда текст

chapters = []
chapters_ = []

errors = []

k = 0
for iter_chapter in chapter_link:
    time.sleep(1)
    #print(iter_chapter)
    
    req_chapter_link = requests.get(iter_chapter)
    chapter_html_text = req_chapter_link.text
    chapter_soup = BeautifulSoup(chapter_html_text,'html.parser')    
    chapter_soup_text = str(chapter_html_text)
     
    # вырезаю split-ом из-за того, что при "find_all" существеная часть текста пропадает
    # Сразу чистим от </p>
    chapter_soup_text = chapter_soup_text.replace("</p>", "")
    
    chapter_soup_text = chapter_soup_text.replace("<strong><u>", "")
    chapter_soup_text = chapter_soup_text.replace("</u></strong> ", "")

    chapter_soup_text = chapter_soup_text.replace("<h4>", "")
    chapter_soup_text = chapter_soup_text.replace("</h4>", "")

    chapter_soup_text = chapter_soup_text.replace("<b></b> ", "")

    chapter_soup_text = chapter_soup_text.replace("</div>", "")
    
    chapter_soup_text2 = chapter_soup_text.split("<div id=\"article\">")
    
    if len(chapter_soup_text2)>1:
        chapter_soup_text3 = chapter_soup_text2[1].split("<div class=\"notice-text\">")
        chapter_soup_text4 = chapter_soup_text3[0].split("<p>")
        ch_er = "Выполнение главы:" + iter_chapter
        errors.append(ch_er)
    else:
        print(iter_chapter)
        print(req_chapter_link)
        chapter_soup_text3 = ""
        chapter_soup_text4 = []
        ch_er = "!!! Проблемная глава:" + iter_chapter
        errors.append(ch_er)
    

    # вырезаем из строк типовое барахло "A day passed. <div style="margin-top: 0px; margin-botto%"
    new_txt = []
    for txt in chapter_soup_text4:
        txt_ = txt.split("<div style")

        # убираем пустые значения из списка абзацев в главе
        if txt_[0]!= "":
                 if txt != " ":
                     new_txt.append(txt_[0])
    
    chapters.append(new_txt)
    update_progress(k / len(chapter_link))
    print("Последняя глава:" + iter_chapter)
    k=k+1



Progress: [####################] 99.5%
Последняя глава:https://freewebnovel.com/invincible-novel/chapter-3753.html


In [ ]:
book = FictionBook2()

title = 'INVINCIBLE'
book.titleInfo.title = title
book.documentInfo.version = "1.1"

book.chapters = []

for i in chapters:
    book.chapters.append((i[0], i[1:]))

book.write(title + ".fb2")